---
title: Deep Space Climate Observatory (DSCOVR)
---

The routines in this module can be used to load data from the Deep Space Climate Observatory (DSCOVR) mission.

## Instruments

- Magnetometer
- Faraday cup


In [ ]:
# | default_exp missions/dscovr/__init__
# | export
from typing import Literal
from space_analysis.core import Instrument, Mission

In [ ]:
# | export
faraday_cup = Instrument(name="Faraday Cup")
mag = Instrument(name="Magnetometer")
dscovr = Mission(name="DSCOVR", instruments=[mag, faraday_cup])

MagDataVars = Literal[
    "B1F1",
    "B1GSE",
    "B1RTN",
]

PlasmaDataVars = Literal[
    "Np",  # Solar Wind Proton Number Density
    "V_GSE",
    "THERMAL_SPD",
    "THERMAL_TEMP",
]

In [ ]:
import pyspedas
import polars as pl
from functools import partial
from typing import Literal, overload
from space_analysis.io.cdf import cdf2pl

In [ ]:
def download_data(**kwargs):
    return pyspedas.dscovr.load(downloadonly=True, **kwargs)


@overload
def load_data(
    instrument: Literal["mag"],
    var_names: list[MagDataVars],
    datatype: Literal["h0"],
    **kwargs,
): ...


@overload
def load_data(
    instrument: Literal["faraday_cup"],
    var_names: list[PlasmaDataVars],
    datatype: Literal["h1"],
    **kwargs,
): ...


def load_data(
    instrument="mag", var_names=None, datatype=None, **kwargs
) -> pl.LazyFrame:
    if instrument == "mag":
        var_names = var_names or ["B1F1", "B1RTN"]
        datatype = datatype or "h0"
    elif instrument == "faraday_cup":
        var_names = var_names or ["Np", "V_GSE", "THERMAL_TEMP"]
        datatype = datatype or "h1"

    files = download_data(instrument=instrument, datatype=datatype, **kwargs)

    load_func = partial(cdf2pl, var_names=var_names)

    return pl.concat(load_func(file) for file in files)